In [0]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.models import model_from_json, load_model
from sklearn.model_selection import train_test_split
# from tensorflow.keras.utils.np_utils import to_categorical
import re

In [0]:
# data = pd.read_csv('/gdrive/My Drive/fgai/DataLabelD.csv')
data = pd.read_excel('/gdrive/My Drive/NLP/Tugas FG/Tugas 1/hehe.xlsx', usecols=[0,1])
# data = data[['Product','Consumer complaint narrative']]
data = data.dropna()

In [0]:
data

,tmessage,Issue
0,no telp internet saya lambat,Komplain layanan Internet Lambat
1,chadijah kompleks griya fajar mas blok e no,Komplain layanan Internet Lambat
2,nah saya mau tanya pemakaian saya kan over seh...,Komplain layanan Internet Lambat
3,sdh saya restart,Komplain layanan Internet Lambat
4,skrng sdh tgl dan belum normal,Komplain layanan Internet Lambat
5,ping download speed mbps biasanya atau,Komplain layanan Internet Lambat
6,muh pahlevi YAHOO,Komplain layanan Internet Lambat
7,jadi saya tunggu petugas kerumah saya atau bgmn,Komplain layanan Internet Lambat
8,perbaikan jaringan sentral makan waktu brp lama,Komplain layanan Internet Lambat
9,prmisi ni kok internet nya lambat,Komplain layanan Internet Lambat


In [0]:
# data['DATA'] = data['DATA'].apply(lambda x: x.lower())
# data['DATA'] = data['DATA'].apply((lambda x: re.sub('[^a-zA-z\s]','',x)))

In [0]:
# labels = ['Informasi tagihan nomor FIRAHOME', 'Koneksi Putus']
labels = ['Migrasi', 'Informasi laporan Isolir', 'Komplain layanan Internet Lambat']

In [0]:
for label in labels:
    print(label)
    print(data[data['Issue']==label].shape)

Migrasi
(1339, 2)
Informasi laporan Isolir
(604, 2)
Komplain layanan Internet Lambat
(63, 2)


In [0]:
data

,tmessage,Issue
0,trs apa isolirnya langsung bsa di buka mba,Informasi laporan Isolir
1,untuk proses migrasi paketnya brp lama ya,Migrasi
2,mau imigrasi speedy keFIRAHOME,Migrasi
3,speedy,Migrasi
4,abd rahman jl bawasalo segeri,Migrasi
5,tanjonge segeri atau dekat kantor lurah segeri...,Migrasi
6,kalau gor belakang puskesmas segeri,Migrasi
7,atau ketimur topram murah tanjonge pas di situ...,Migrasi
8,kapan bisa aktip FIRAHOME di sini sudah lama m...,Migrasi
9,sudah lama menunggu kapan sudah melakukan vote...,Migrasi


In [0]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['tmessage'].values)
X = tokenizer.texts_to_sequences(data['tmessage'].values)
X = pad_sequences(X)

In [0]:
print(X.shape)

(2006, 160)


In [0]:
embed_dim = 32
lstm_out = 100

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 160, 32)           64000     
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 160, 32)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 3)                 303       
Total params: 117,503
Trainable params: 117,503
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
Y = pd.get_dummies(data['Issue']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
print(Y_train[0])

(1344, 160) (1344, 3)
(662, 160) (662, 3)
[0 0 1]


In [0]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 8, batch_size=batch_size)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/8
1344/1344 [==============================] - 18s 13ms/sample - loss: 0.8335 - acc: 0.6533
Epoch 2/8
1344/1344 [==============================] - 16s 12ms/sample - loss: 0.6675 - acc: 0.6830
Epoch 3/8
1344/1344 [==============================] - 17s 12ms/sample - loss: 0.3646 - acc: 0.8958
Epoch 4/8
1344/1344 [==============================] - 17s 12ms/sample - loss: 0.1855 - acc: 0.9427
Epoch 5/8
1344/1344 [==============================] - 16s 12ms/sample - loss: 0.1345 - acc: 0.9524
Epoch 6/8
1344/1344 [==============================] - 16s 12ms/sample - loss: 0.1098 - acc: 0.9576
Epoch 7/8
1344/1344 [==============================] - 17s 12ms/sample - loss: 0.0895 - acc: 0.9621
Epoch 8/8
1344/1344 [==============================] - 17s 12ms/sample - loss: 0.0831 - acc: 0.9643


In [0]:
validation_size = 30

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

632/632 - 3s - loss: 0.1094 - acc: 0.9668
score: 0.11
acc: 0.97


In [0]:
model.save("3label.h5")
print("Saved model")

Saved model


In [0]:
from google.colab import files
files.download('3label.h5')

MessageError: ignored

In [0]:
model = load_model("3label.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 160, 32)           64000     
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 160, 32)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 3)                 303       
Total params: 117,503
Trainable params: 117,503
Non-trainable params: 0
_________________________________________________________________


In [0]:
kalimat = ["gimana mau migrasi datanya"]
test = tokenizer.texts_to_sequences(kalimat)
test = pad_sequences(test, maxlen=X.shape[1])

In [0]:
vector = np.array([test.flatten()])
model.predict_classes(vector)

array([2])

In [0]:
con_data = np.asarray(data['tmessage'])

In [0]:
con_data

array(['no telp internet saya lambat',
       'chadijah kompleks griya fajar mas blok e no',
       'nah saya mau tanya pemakaian saya kan over sehingga speednya turun dan akan normal tgl nah skrng sdh tgl tp belum normal',
       ...,
       'tapi sejak hari jumat saya buat laporannya sampai sekarang belum jga aktif pak tapi kok sebntar ini saya melapor bilang csnya jaringan saya direstart lalu saya coba sempat bisa slama menit tapi isolir lagi bilangnya',
       'migrasi paket',
       'internet saya sampai sekarg belum aktif buk saya cek masi terisolir udah berapa kali saya melapor tpi tidak kunjung aktif kasusnya kemaren saya minta putus telepon tapi malah diputus semua'],
      dtype=object)

In [0]:
suku_kata = pd.read_excel('/gdrive/My Drive/NLP/Tugas FG/Daftar Kata+Silabel Indihome.xlsx')

In [0]:
kata_dasar = suku_kata['kata']
kata_suku = suku_kata['ka.ta']

In [0]:
new_word = {}
for i in range(len(kata_dasar)):
  temp = str(kata_dasar[i])
  temp_kata = str(kata_suku[i])
  new_word[temp] = temp_kata

In [0]:
pip install nltk

In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
import nltk
clean_data = []
for i in con_data:
  clean_data.append(nltk.word_tokenize(i))

In [0]:
test = clean_data[:200]
test

[['no', 'telp', 'internet', 'saya', 'lambat'],
 ['chadijah', 'kompleks', 'griya', 'fajar', 'mas', 'blok', 'e', 'no'],
 ['nah',
  'saya',
  'mau',
  'tanya',
  'pemakaian',
  'saya',
  'kan',
  'over',
  'sehingga',
  'speednya',
  'turun',
  'dan',
  'akan',
  'normal',
  'tgl',
  'nah',
  'skrng',
  'sdh',
  'tgl',
  'tp',
  'belum',
  'normal'],
 ['sdh', 'saya', 'restart'],
 ['skrng', 'sdh', 'tgl', 'dan', 'belum', 'normal'],
 ['ping', 'download', 'speed', 'mbps', 'biasanya', 'atau'],
 ['muh', 'pahlevi', 'YAHOO'],
 ['jadi', 'saya', 'tunggu', 'petugas', 'kerumah', 'saya', 'atau', 'bgmn'],
 ['perbaikan', 'jaringan', 'sentral', 'makan', 'waktu', 'brp', 'lama'],
 ['prmisi', 'ni', 'kok', 'internet', 'nya', 'lambat'],
 ['internetnya', 'lambat'],
 ['kenapa', 'internet', 'saya', 'lambat', 'sekali'],
 ['no', 'intenet'],
 ['belom'],
 ['speednya', 'turun'],
 ['baik', 'terima', 'kasih'],
 ['ya', 'benar'],
 ['tadi', 'saya', 'tes', 'sekitar'],
 ['tes', 'berikutnya', 'dibawah'],
 ['saya', 'tes', 'ul

In [0]:
def cek_sillabels(suku_kata, kata):
  for i in suku_kata:
    if i[0] == kata:
      return i[1]
    else:
      pass

In [0]:
for kalimat in clean_data:
  for i, kata in enumerate(kalimat):
    if kata in new_word.keys():
      kalimat[i] = new_word[kata]
#       print(new_word[kata])
#       print(clean_data[kalimat][kata])

In [0]:
new_word['kecewa']

'ke.ce.wa'

In [0]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
TreebankWordDetokenizer().detokenize(clean_data[0])

'no telp in.ter.net saya lam.bat'

In [0]:
s = '.'
s = s.join(clean_data[0]) 
print(s)

no.telp.in.ter.net.saya.lam.bat


In [0]:
for i, k in enumerate(clean_data):
  s = '.'
  s = s.join(clean_data[i]) 
#   s = s.split('.')
  clean_data[i] = s

In [0]:
clean_data

['no.telp.in.ter.net.saya.lam.bat',
 'cha.di.jah.kompleks.gri.ya.fa.jar.mas.blok.e.no',
 'nah.saya.mau.tanya.pe.ma.kai.an.saya.kan.o.ver.sehingga.spe.ed.nya.tu.run.dan.akan.nor.mal.tgl.nah.skrng.sdh.tgl.tp.belum.nor.mal',
 'sdh.saya.re.start',
 'skrng.sdh.tgl.dan.belum.nor.mal',
 'ping.do.wnlo.ad.spe.ed.mbps.biasanya.atau',
 'muh.pah.le.vi.YAHOO',
 'jadi.saya.tung.gu.pe.tu.gas.ke.ru.mah.saya.atau.bgmn',
 'per.bai.kan.ja.ri.ngan.sen.tral.ma.kan.waktu.brp.lama',
 'prmi.si.ni.kok.in.ter.net.nya.lam.bat',
 'in.ter.net.nya.lam.bat',
 'kenapa.in.ter.net.saya.lam.bat.sekali',
 'no.in.te.net',
 'be.lom',
 'spe.ed.nya.tu.run',
 'baik.te.ri.ma.kasih',
 'ya.benar',
 'tadi.saya.tes.sekitar',
 'tes.berikutnya.di.ba.wah',
 'saya.tes.u.lang.ya',
 'http.be.ta.spe.e.dtest.net.re.sult',
 'baik.te.ri.ma.kasih',
 'tang.kas.ge.de.YAHOO',
 'baik.te.ri.ma.kasih',
 'ski.tar.lebih',
 'lan.nya.la.ber.ke.dip.wlan.nya.la.ber.ke.dip',
 'eng.ga',
 'ma.sih.sama.spe.ed.nya',
 'ba.ru.san.sy.cek.di.spe.e.dtest.net',
 '

In [0]:
for i, k in enumerate(clean_data):
  clean_data[i] = clean_data[i].split('.')

In [0]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(clean_data)
X = tokenizer.texts_to_sequences(clean_data)
X = pad_sequences(X)

In [0]:
X

array([[  0,   0,   0, ...,   4, 167, 291],
       [  0,   0,   0, ..., 785,  70,  48],
       [  0,   0,   0, ...,  62, 362, 309],
       ...,
       [  0,   0,   0, ...,  60,  66,  12],
       [  0,   0,   0, ...,   1,   9,  13],
       [  0,   0,   0, ..., 212, 146, 347]], dtype=int32)

In [0]:
embed_dim = 32
lstm_out = 100

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 274, 32)           64000     
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 274, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 303       
Total params: 117,503
Trainable params: 117,503
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
Y = pd.get_dummies(data['Issue']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
print(Y_train[0])

(1344, 274) (1344, 3)
(662, 274) (662, 3)
[0 0 1]


In [0]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 8, batch_size=batch_size)

Epoch 1/8
1344/1344 [==============================] - 29s 21ms/sample - loss: 0.8299 - acc: 0.6518
Epoch 2/8
1344/1344 [==============================] - 28s 21ms/sample - loss: 0.4624 - acc: 0.8348
Epoch 3/8
1344/1344 [==============================] - 28s 21ms/sample - loss: 0.2107 - acc: 0.9301
Epoch 4/8
1344/1344 [==============================] - 28s 21ms/sample - loss: 0.1421 - acc: 0.9487
Epoch 5/8
1344/1344 [==============================] - 28s 21ms/sample - loss: 0.1311 - acc: 0.9524
Epoch 6/8
1344/1344 [==============================] - 28s 21ms/sample - loss: 0.0888 - acc: 0.9613
Epoch 7/8
1344/1344 [==============================] - 28s 21ms/sample - loss: 0.0921 - acc: 0.9673
Epoch 8/8
1344/1344 [==============================] - 28s 21ms/sample - loss: 0.0619 - acc: 0.9784


In [0]:
validation_size = 30

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

632/632 - 5s - loss: 0.0652 - acc: 0.9731
score: 0.07
acc: 0.97


In [0]:
model.save("model_sukukata_3label.h5")
print("Saved model")

Saved model


In [0]:
from google.colab import files
files.download('model_sukukata.h5')

In [0]:
model = load_model("model_sukukata_3label.h5")

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 274, 32)           64000     
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 274, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 303       
Total params: 117,503
Trainable params: 117,503
Non-trainable params: 0
_________________________________________________________________


In [0]:
for label in labels:
    print(label)
    print(data[data['Issue']==label].shape)

Migrasi
(1339, 2)
Informasi laporan Isolir
(604, 2)
Komplain layanan Internet Lambat
(63, 2)


In [0]:
new_word['gimana']

In [0]:
kalimat = ['coba dong gimana migrasinya']

In [0]:
kalimat = kalimat[0].split(" ")
for i, kata in enumerate(kalimat):
  if kata in new_word.keys():
      kalimat[i] = new_word[kata]
s = '.'
s = s.join(kalimat) 

In [0]:
s

'coba.dong.gi.ma.na.mi.gra.si.nya'

In [0]:
kalimat = [s]
test = tokenizer.texts_to_sequences(kalimat)
test = pad_sequences(test, maxlen=X.shape[1])

In [0]:
test

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0

In [0]:
model.predict(test)

array([[3.8955134e-04, 2.9897848e-03, 9.9662066e-01]], dtype=float32)

In [0]:
save = model.predict(test)

In [0]:
val = max(save[0])
# np.where(save[0] == val)[0][0]
labels[np.where(save[0] == val)[0][0]]

2